In [1]:
import pandas as pd
import numpy as np
import os
import pickle
import csv

In [2]:
filename = 'LD2011_2014.txt'

df = pd.read_csv(filename, index_col=0, sep=';', decimal=',')
df.index = pd.to_datetime(df.index)
feature_names = df.columns.tolist()
feature_num = len(feature_names)
df['datetime'] = pd.to_datetime(df.index)
print(df)

                       MT_001     MT_002    MT_003      MT_004     MT_005  \
2011-01-01 00:15:00  0.000000   0.000000  0.000000    0.000000   0.000000   
2011-01-01 00:30:00  0.000000   0.000000  0.000000    0.000000   0.000000   
2011-01-01 00:45:00  0.000000   0.000000  0.000000    0.000000   0.000000   
2011-01-01 01:00:00  0.000000   0.000000  0.000000    0.000000   0.000000   
2011-01-01 01:15:00  0.000000   0.000000  0.000000    0.000000   0.000000   
...                       ...        ...       ...         ...        ...   
2014-12-31 23:00:00  2.538071  22.048364  1.737619  150.406504  85.365854   
2014-12-31 23:15:00  2.538071  21.337127  1.737619  166.666667  81.707317   
2014-12-31 23:30:00  2.538071  20.625889  1.737619  162.601626  82.926829   
2014-12-31 23:45:00  1.269036  21.337127  1.737619  166.666667  85.365854   
2015-01-01 00:00:00  2.538071  19.914651  1.737619  178.861789  84.146341   

                         MT_006     MT_007      MT_008     MT_009     MT_01

In [6]:
print(f'Original df missing rate: '
            f'{(df.loc[:, feature_names].isna().sum().sum() / (df.shape[0] * df.shape[1])):.3f}')
mask = ~(df.loc[:, feature_names].isna().values)
print(df.shape)
print(mask.shape)
print(mask)

Original df missing rate: 0.000
(140256, 371)
(140256, 370)
[[ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 ...
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]]


In [7]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x = scaler.fit_transform(df.loc[:, feature_names])


def window_truncate(feature_vectors, seq_len):
    """ Generate time series samples, truncating windows from time-series data with a given sequence length.
    Parameters
    ----------
    feature_vectors: time series data, len(shape)=2, [total_length, feature_num]
    seq_len: sequence length
    """
    start_indices = np.asarray(range(feature_vectors.shape[0] // seq_len)) * seq_len
    sample_collector = []
    for idx in start_indices:
        sample_collector.append(feature_vectors[idx: idx + seq_len])

    return np.asarray(sample_collector).astype('float32')


seq_len = 100
x = window_truncate(x, seq_len)
x[np.isnan(x)] = 0
mask = window_truncate(mask, seq_len)
# print(x)
# print(mask)
print(x.shape, mask.shape)

(1402, 100, 370) (1402, 100, 370)


In [8]:
pickle.dump((x, mask), open('data.pkl', 'wb'))